In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas.io import sql
import snowflake.connector
import keyring
import psycopg2 
import time
from datetime import date, timedelta
from scipy import stats

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 500)

from matplotlib import pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
sns.set_style("darkgrid")

In [3]:
snowflake_username = 'matthew.bessey@disneystreaming.com'

In [4]:
ctx = snowflake.connector.connect(authenticator='externalbrowser', 
                                  user=snowflake_username, 
                                  account='disneystreaming.us-east-1')

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [5]:
# set date parameters for query

subscription_start_date_min = "'2019-11-12'"
subscription_start_date_max = "'2019-11-13'" # max of subscription start date
engagement_date = "'2019-12-19'" # date for which we want to pull engagement behaviors

In [6]:
query= """
select o.swid
, o.swid_holdout
, a.accountid
, s.subscription_id
, e.*
from subscription s
join account a on s.account_id = a.accountid
join oneid_combined o on a.swid = o.swid
join "DSS_PROD"."DISNEY_PLUS"."DIM_DISNEY_DAILY_ACCOUNT_ENGAGEMENT" e on a.accountid = e.account_id
where s.partner = 'disney'
and s.calculated_subscription_start_dtm >= {}
and s.CALCULATED_SUBSCRIPTION_START_DTM <= {}
--and s.is_entitled = 1
and e.ds = {}
and e.is_pre_launch != 1
limit 100000;
""".format(subscription_start_date_min, subscription_start_date_max, engagement_date)

In [7]:
# run the query and write to engagement
engagement = pd.read_sql(query,ctx)

In [8]:
# map columns to lowercase
engagement.columns = engagement.columns.str.lower()

In [9]:
# create function and apply for mapping of holdout groups on 'swid_holdout'
def holdout_grouping(df):
    if df['swid_holdout'] < 243:
        return "all marketing"
    elif df['swid_holdout'] >= 243 and df['swid_holdout'] < 246:
        return "no onboarding"
    else:
        return "no marketing"
    
engagement['marketing_holdout'] = engagement.apply(holdout_grouping,axis=1)

In [10]:
#drop rows w/ no entitlement data, rewrite as int
engagement = engagement.drop(engagement[engagement['is_entitled']=='unknown'].index,axis=0)
engagement.is_entitled = engagement.is_entitled.astype(int)

In [16]:
# remove unnecessary columns
columnsToRemove = [        'swid',
    'swid_holdout',
    'accountid',
    'subscription_id',
    'ds',
    'account_id',
    'is_flagged',
    'first_account_subscription_signup_week',
    'last_account_subscription_signup_week',
    'is_entitled_l1',
    'is_entitled_l7',
    'is_entitled_l28',
    'is_entitled_itd',
    'is_pre_launch',
    'is_pre_launch_nltt',
    'account_home_country',
    'subscription_state_upd',
    'subscription_type',
    'total_login_days_l1',
    'total_login_days_l28',
    'total_streams_l1',
    'total_streams_l28',
    'num_streaming_profiles_l1',
    'num_streaming_profiles_l28',
    'num_general_streaming_profiles_l1',
    'num_general_streaming_profiles_l28',
    'num_kids_streaming_profiles_l1',
    'num_kids_streaming_profiles_l28',
    'num_streaming_devices_l1',
    'num_streaming_devices_l28',
    'account_total_stream_days_l1',
    'account_total_stream_days_l28',
    'account_profile_total_stream_days_l1',
    'account_profile_total_stream_days_l28',
    'total_stream_days_general_profiles_l1',
    'total_stream_days_general_profiles_l28',
    'total_stream_days_kids_profiles_l1',
    'total_stream_days_kids_profiles_l28',
    'total_stream_time_ms_l1',
    'total_stream_time_ms_l28',
    'total_stream_time_general_profiles_ms_l1',
    'total_stream_time_general_profiles_ms_l28',
    'total_stream_time_kids_profiles_ms_l1',
    'total_stream_time_kids_profiles_ms_l28',
    'total_stream_time_web_ms_l1',
    'total_stream_time_web_ms_l28',
    'total_stream_time_mobile_ms_l1',
    'total_stream_time_mobile_ms_l28',
    'total_stream_time_connected_tv_ms_l1',
    'total_stream_time_connected_tv_ms_l28',
    'total_stream_time_unknown_ms_l1',
    'total_stream_time_unknown_ms_l28',
    'last_stream_date',
    'account_profile_total_stream_days_l7',
    'account_profile_total_stream_days_itd',
    'total_stream_time_unknown_ms_l7',
    'total_stream_time_unknown_ms_itd',
]

engagementCleaned = engagement.drop(columnsToRemove,axis=1)

In [19]:
renamedColumns = [
        'first_signup_date',
    'last_signup_date', 
    'is_entitled',
    'ttl_login_days_l7', 
    'ttl_login_days_itd', 
    'ttl_streams_l7',
    'ttl_streams_itd', 
    'streaming_profiles_l7',
    'streaming_profiles_itd', 
    'general_streaming_profiles_l7',
    'general_streaming_profiles_itd', 
    'kids_streaming_profiles_l7',
    'kids_streaming_profiles_itd', 
    'streaming_devices_l7',
    'streaming_devices_itd', 
    'ttl_stream_days_l7',
    'ttl_stream_days_itd',
    'ttl_stream_days_general_profiles_l7',
    'ttl_stream_days_general_profiles_itd',
    'ttl_stream_days_kids_profiles_l7',
    'ttl_stream_days_kids_profiles_itd', 
    'ttl_stream_time_ms_l7',
    'ttl_stream_time_ms_itd', 
    'ttl_stream_time_general_profiles_ms_l7',
    'ttl_stream_time_general_profiles_ms_itd',
    'ttl_stream_time_kids_profiles_ms_l7',
    'ttl_stream_time_kids_profiles_ms_itd', 
    'ttl_stream_time_web_ms_l7',
    'ttl_stream_time_web_ms_itd', 
    'ttl_stream_time_mobile_ms_l7',
    'ttl_stream_time_mobile_ms_itd',
    'ttl_stream_time_connected_tv_ms_l7',
    'tl_stream_time_connected_tv_ms_itd', 
    'days_since_last_stream',
    'marketing_holdout'
]

engagementCleaned.columns = renamedColumns

In [37]:
# remove non-US countries and then country column
countryUS_filter = engagement['account_home_country'] == 'US'
engagementCleaned = engagementCleaned[countryUS_filter]
engagementCleaned = engagementCleaned.drop('account_home_country',axis=1)

In [21]:
# rename for conciseness
engmt = engagementCleaned
del engagementCleaned
del engagement

In [22]:
engmt.columns

Index(['first_signup_date', 'last_signup_date', 'is_entitled',
       'ttl_login_days_l7', 'ttl_login_days_itd', 'ttl_streams_l7',
       'ttl_streams_itd', 'streaming_profiles_l7', 'streaming_profiles_itd',
       'general_streaming_profiles_l7', 'general_streaming_profiles_itd',
       'kids_streaming_profiles_l7', 'kids_streaming_profiles_itd',
       'streaming_devices_l7', 'streaming_devices_itd', 'ttl_stream_days_l7',
       'ttl_stream_days_itd', 'ttl_stream_days_general_profiles_l7',
       'ttl_stream_days_general_profiles_itd',
       'ttl_stream_days_kids_profiles_l7', 'ttl_stream_days_kids_profiles_itd',
       'ttl_stream_time_ms_l7', 'ttl_stream_time_ms_itd',
       'ttl_stream_time_general_profiles_ms_l7',
       'ttl_stream_time_general_profiles_ms_itd',
       'ttl_stream_time_kids_profiles_ms_l7',
       'ttl_stream_time_kids_profiles_ms_itd', 'ttl_stream_time_web_ms_l7',
       'ttl_stream_time_web_ms_itd', 'ttl_stream_time_mobile_ms_l7',
       'ttl_stream_time_mobi

In [28]:
# define columns for binary construction
# binary = 1 if action occurred in interval, else 0

binaryConstructionList = [
    'ttl_login_days_l7',
    'ttl_login_days_itd',
    'ttl_streams_l7',
    'ttl_streams_itd',
    'general_streaming_profiles_l7', 'general_streaming_profiles_itd',
    'kids_streaming_profiles_l7', 'kids_streaming_profiles_itd',
    'ttl_stream_time_web_ms_l7',
    'ttl_stream_time_web_ms_itd', 
    'ttl_stream_time_mobile_ms_l7',
    'ttl_stream_time_mobile_ms_itd', 
    'ttl_stream_time_connected_tv_ms_l7',
    'tl_stream_time_connected_tv_ms_itd', 
    'days_since_last_stream'
]

In [29]:
# create binary variables _bin
for i in binaryConstructionList:
    engmt[i + '_bin'] = engmt.apply(lambda df:
                                   1 if df[i] > 0
                                   else 0,
                                   axis=1)

In [32]:
engmt.groupby('marketing_holdout').mean()

,is_entitled,ttl_login_days_l7,ttl_login_days_itd,ttl_streams_l7,ttl_streams_itd,streaming_profiles_l7,streaming_profiles_itd,general_streaming_profiles_l7,general_streaming_profiles_itd,kids_streaming_profiles_l7,kids_streaming_profiles_itd,streaming_devices_l7,streaming_devices_itd,ttl_stream_days_l7,ttl_stream_days_itd,ttl_stream_days_general_profiles_l7,ttl_stream_days_general_profiles_itd,ttl_stream_days_kids_profiles_l7,ttl_stream_days_kids_profiles_itd,ttl_stream_time_ms_l7,ttl_stream_time_ms_itd,ttl_stream_time_general_profiles_ms_l7,ttl_stream_time_general_profiles_ms_itd,ttl_stream_time_kids_profiles_ms_l7,ttl_stream_time_kids_profiles_ms_itd,ttl_stream_time_web_ms_l7,ttl_stream_time_web_ms_itd,ttl_stream_time_mobile_ms_l7,ttl_stream_time_mobile_ms_itd,ttl_stream_time_connected_tv_ms_l7,tl_stream_time_connected_tv_ms_itd,days_since_last_stream,ttl_login_days_l7_bin,ttl_login_days_itd_bin,ttl_streams_l7_bin,ttl_streams_itd_bin,general_streaming_profiles_l7_bin,general_streaming_profiles_itd_bin,kids_streaming_profiles_l7_bin,kids_streaming_profiles_itd_bin,ttl_stream_time_web_ms_l7_bin,ttl_stream_time_web_ms_itd_bin,ttl_stream_time_mobile_ms_l7_bin,ttl_stream_time_mobile_ms_itd_bin,ttl_stream_time_connected_tv_ms_l7_bin,tl_stream_time_connected_tv_ms_itd_bin,days_since_last_stream_bin
marketing_holdout,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
all marketing,0.906883,4.959158,28.838212,18.351615,128.692915,1.339118,1.952467,1.259868,1.827464,0.079745,0.136720,1.990285,4.135751,3.427715,21.190723,4.045816,25.439812,0.240901,1.432225,3.162180e+07,2.262726e+08,2.996404e+07,2.160289e+08,1.657756e+06,1.024364e+07,0.0,0.0,0.0,0.0,0.0,0.0,4.610321,0.895218,1.0,0.822333,0.985174,0.816569,0.984911,0.068650,0.106405,0.0,0.0,0.0,0.0,0.0,0.0,0.515216
no marketing,0.913883,5.019050,29.196242,18.852035,131.498695,1.342119,1.928497,1.264092,1.808455,0.079071,0.131002,1.979123,4.163361,3.474165,21.518528,4.074896,25.685543,0.233559,1.421451,3.294250e+07,2.360479e+08,3.129439e+07,2.250942e+08,1.648116e+06,1.095366e+07,0.0,0.0,0.0,0.0,0.0,0.0,4.349522,0.906315,1.0,0.831681,0.983299,0.825418,0.983038,0.068633,0.104384,0.0,0.0,0.0,0.0,0.0,0.0,0.521921
no onboarding,0.910084,4.966387,28.817647,19.734454,134.657143,1.390756,1.945378,1.307563,1.815966,0.084874,0.141176,2.093277,4.242857,3.542857,21.507563,4.147059,25.568067,0.255462,1.575630,3.395392e+07,2.368546e+08,3.218076e+07,2.255731e+08,1.773161e+06,1.128148e+07,0.0,0.0,0.0,0.0,0.0,0.0,4.119352,0.901681,1.0,0.853782,0.985714,0.846218,0.985714,0.074790,0.106723,0.0,0.0,0.0,0.0,0.0,0.0,0.497479


In [33]:
(engmt.groupby('marketing_holdout').mean().iloc[0,]
 /engmt.groupby('marketing_holdout').mean().iloc[1,]-1)*100

is_entitled                               -0.765984
ttl_login_days_l7                         -1.193298
ttl_login_days_itd                        -1.226288
ttl_streams_l7                            -2.654462
ttl_streams_itd                           -2.133694
streaming_profiles_l7                     -0.223583
streaming_profiles_itd                     1.242931
general_streaming_profiles_l7             -0.334136
general_streaming_profiles_itd             1.051118
kids_streaming_profiles_l7                 0.852956
kids_streaming_profiles_itd                4.364737
streaming_devices_l7                       0.563964
streaming_devices_itd                     -0.663181
ttl_stream_days_l7                        -1.337013
ttl_stream_days_itd                       -1.523362
ttl_stream_days_general_profiles_l7       -0.713636
ttl_stream_days_general_profiles_itd      -0.956691
ttl_stream_days_kids_profiles_l7           3.143371
ttl_stream_days_kids_profiles_itd          0.757950
ttl_stream_t

In [16]:
engagement.groupby('marketing_holdout').median()

,swid_holdout,is_flagged,is_entitled,is_pre_launch,is_pre_launch_nltt,total_login_days_l1,total_login_days_l7,total_login_days_l28,total_login_days_itd,total_streams_l1,total_streams_l7,total_streams_l28,total_streams_itd,num_streaming_profiles_l1,num_streaming_profiles_l7,num_streaming_profiles_l28,num_streaming_profiles_itd,num_general_streaming_profiles_l1,num_general_streaming_profiles_l7,num_general_streaming_profiles_l28,num_general_streaming_profiles_itd,num_kids_streaming_profiles_l1,num_kids_streaming_profiles_l7,num_kids_streaming_profiles_l28,num_kids_streaming_profiles_itd,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_devices_l28,num_streaming_devices_itd,account_total_stream_days_l1,account_total_stream_days_l7,account_total_stream_days_l28,account_total_stream_days_itd,account_profile_total_stream_days_l1,account_profile_total_stream_days_l7,account_profile_total_stream_days_l28,account_profile_total_stream_days_itd,total_stream_days_general_profiles_l1,total_stream_days_general_profiles_l7,total_stream_days_general_profiles_l28,total_stream_days_general_profiles_itd,total_stream_days_kids_profiles_l1,total_stream_days_kids_profiles_l7,total_stream_days_kids_profiles_l28,total_stream_days_kids_profiles_itd,total_stream_time_ms_l1,total_stream_time_ms_l7,total_stream_time_ms_l28,total_stream_time_ms_itd,total_stream_time_general_profiles_ms_l1,total_stream_time_general_profiles_ms_l7,total_stream_time_general_profiles_ms_l28,total_stream_time_general_profiles_ms_itd,total_stream_time_kids_profiles_ms_l1,total_stream_time_kids_profiles_ms_l7,total_stream_time_kids_profiles_ms_l28,total_stream_time_kids_profiles_ms_itd,total_stream_time_web_ms_l1,total_stream_time_web_ms_l7,total_stream_time_web_ms_l28,total_stream_time_web_ms_itd,total_stream_time_mobile_ms_l1,total_stream_time_mobile_ms_l7,total_stream_time_mobile_ms_l28,total_stream_time_mobile_ms_itd,total_stream_time_connected_tv_ms_l1,total_stream_time_connected_tv_ms_l7,total_stream_time_connected_tv_ms_l28,total_stream_time_connected_tv_ms_itd,total_stream_time_unknown_ms_l1,total_stream_time_unknown_ms_l7,total_stream_time_unknown_ms_l28,total_stream_time_unknown_ms_itd,days_since_last_stream,active_binary_l1,active_binary_l7
marketing_holdout,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
all marketing,121.0,0.0,1.0,0.0,0.0,1.0,6.0,24.0,33.0,0.0,8.0,47.0,84.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,3.0,15.0,22.0,0.0,4.0,16.0,24.0,0.0,3.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,16736881.0,93714004.0,159986427.0,0.0,15475739.0,87798517.0,151160071.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
no marketing,250.0,0.0,1.0,0.0,0.0,1.0,6.0,24.0,34.0,0.0,8.0,47.0,83.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,3.0,15.0,22.0,0.0,4.0,16.0,24.0,0.0,3.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,16588688.0,93034680.0,157585103.0,0.0,15309058.0,87018730.0,148728338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
no onboarding,244.0,0.0,1.0,0.0,0.0,1.0,6.0,24.0,34.0,0.0,8.0,49.0,84.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,3.0,15.0,22.0,0.0,4.0,16.0,24.0,0.0,3.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,16986927.5,95372095.0,162220157.0,0.0,15863956.5,88671636.5,153850053.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [16]:

#q1 = stats.ttest_ind(engagement.loc[engagement['marketing_holdout']=='all marketing','is_entitled'],
 #                       engagement.loc[engagement.marketing_holdout=='no onboarding','is_entitled'])[1]

#print(q1)


In [30]:
#slice df for marketing and holdout slices

marketing_slice = engagement[engagement['marketing_holdout'] == 'all marketing']
holdout_slice = engagement[engagement['marketing_holdout'] == 'no marketing']

In [32]:
p_values = []

for i in range(16,len(marketing_slice.columns)):
    try: 
        p1 = stats.ttest_ind(marketing_slice.iloc[:,i],
                        holdout_slice.iloc[:,i])[1]
        descr1 = marketing_slice.columns[i]
    except (TypeError,RuntimeWarning): 
        p1 = "Broke!"
    p_values.append([descr1,p1])
    print("Completed",i)


Completed 16
Completed 17


/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Completed 18
Completed 19
Completed 20
Completed 21
Completed 22
Completed 23
Completed 24
Completed 25
Completed 26
Completed 27
Completed 28
Completed 29
Completed 30
Completed 31
Completed 32
Completed 33
Completed 34
Completed 35
Completed 36
Completed 37
Completed 38
Completed 39
Completed 40
Completed 41
Completed 42
Completed 43
Completed 44
Completed 45
Completed 46
Completed 47
Completed 48
Completed 49
Completed 50
Completed 51
Completed 52
Completed 53
Completed 54
Completed 55
Completed 56
Completed 57
Completed 58
Completed 59
Completed 60
Completed 61
Completed 62
Completed 63
Completed 64
Completed 65
Completed 66
Completed 67
Completed 68
Completed 69
Completed 70
Completed 71
Completed 72
Completed 73
Completed 74
Completed 75
Completed 76
Completed 77
Completed 78
Completed 79
Completed 80
Completed 81
Completed 82
Completed 83
Completed 84
Completed 85
Completed 86
Completed 87
Completed 88
Completed 89
Completed 90
Completed 91


In [34]:
p_values

[['is_pre_launch', nan],
 ['is_pre_launch_nltt', 0.8493168085179987],
 ['is_pre_launch_nltt', 'Broke!'],
 ['is_pre_launch_nltt', 'Broke!'],
 ['is_pre_launch_nltt', 'Broke!'],
 ['total_login_days_l1', 0.12294656565361425],
 ['total_login_days_l7', 0.08797844880950126],
 ['total_login_days_l28', 0.019587342840840027],
 ['total_login_days_itd', 0.010880849218279927],
 ['total_streams_l1', 0.11661642143870479],
 ['total_streams_l7', 0.2660619922234634],
 ['total_streams_l28', 0.11724506396719173],
 ['total_streams_itd', 0.09099968452959808],
 ['num_streaming_profiles_l1', 1.9563725531440613e-05],
 ['num_streaming_profiles_l7', 0.004456732365602317],
 ['num_streaming_profiles_l28', 0.000279954709103938],
 ['num_streaming_profiles_itd', 0.00020447072107953083],
 ['num_general_streaming_profiles_l1', 2.91058212779101e-06],
 ['num_general_streaming_profiles_l7', 0.000868598332836167],
 ['num_general_streaming_profiles_l28', 0.00012502116423895935],
 ['num_general_streaming_profiles_itd', 0.000